# Carregando bibliotecas

In [2]:
import grequests
import pandas as pd
import re
import requests
import warnings
import urllib.parse
import datetime

warnings.simplefilter("ignore", category=UserWarning)

# Definindo Variáveis para usar o API

O método escolhido para extração dos dados foi por consultas diretamente por API

In [3]:
#Url para chamar a API e escolha dos métodos

base_endpoint = 'https://opendata.nhsbsa.net/api/3/action/'
package_list_method = 'package_list'     # Lista dos datasets no portal
package_show_method = 'package_show?id=' # Lista todos recursos de dataset
action_method = 'datastore_search_sql?'  # método para ação SQL

# 10 substâncias químicas mais prescritas para cada região

### Lista de regiões

A solução encontrada para esse problema, foi separar as regiões e então concatená-las.

In [4]:
#Dataset para o mês 5 do ano de 2022

nome_dataset = 'EPD_202205'

In [5]:
#Criando o query para obter todas regiões no dataset

query_todas_regioes = "SELECT " \
                     "DISTINCT regional_office_name " \
                     f"FROM `{nome_dataset}` "

In [6]:
#Método para codificar o método em uma url

todas_regioes_api_call = f"{base_endpoint}" \
                        f"{action_method}" \
                        "resource_id=" \
                        f"{nome_dataset}" \
                        "&" \
                        "sql=" \
                        f"{urllib.parse.quote(query_todas_regioes)}"

#Resposta JSON
json_regioes = requests.get(todas_regioes_api_call).json()

#Extrai os dados requisitados para um dataframe
todas_regioes_df  = pd.json_normalize(json_regioes['result']['result']['records'])

### Extração e concatenação

Foi necessário dividir as extrações por meses, não é possível extrair todos pelo grande volume de dados.\
Não há acréscimo de regiões em todo o período estudado, por esse motivo não foi feito novas listas de regiões para cada período.\
Nessa extração usou-se as 15 maiores substâncias, para diminuir a possibilidade das 10 substâncias mais prescritas mudarem de mês pra mês.\

##### Extraindo dados do mês 5 do ano de 2022

In [7]:
#Selecionando o mês para extrair os dados
nome_dataset = 'EPD_202205'

#Criação do dataset em que será concatenado
regioes202205_df = pd.DataFrame()

#for loop para extrair cada dataset regional e então concatená-los
for i in range(0, len(todas_regioes_df)) :
   
   #Query para uma região
   query_uma_regiao = "SELECT " \
                           "regional_office_name, " \
                           "chemical_substance_bnf_descr, " \
                           "sum(total_quantity) as quantity " \
                        "FROM " \
                           f"`{nome_dataset}` " \
                        "WHERE " \
                           "regional_office_name = " f"'{todas_regioes_df.regional_office_name[i]}' " \
                        "GROUP BY " \
                           "regional_office_name, " \
                           "chemical_substance_bnf_descr " \
                        "ORDER BY " \
                           "quantity DESC " \
                        "LIMIT 15"

   #criação da url de requisição ao API
   uma_regiao_api_call = f"{base_endpoint}" \
                           f"{action_method}" \
                           "resource_id=" \
                           f"{nome_dataset}" \
                           "&" \
                           "sql=" \
                           f"{urllib.parse.quote(query_uma_regiao)}"

   #Resultado em JSON temporário
   json_uma_regiao = requests.get(uma_regiao_api_call).json()

   #Extração apenas dos resultados do JSON e transformação em um dataframe
   uma_regiao_df  = pd.json_normalize(json_uma_regiao['result']['result']['records'])

   #Concatenação dos dataframes de região
   frames = [regioes202205_df, uma_regiao_df]
   regioes202205_df = pd.concat(frames, sort = False)

#Arrumando o índice
regioes202205_df = regioes202205_df.reset_index(drop = True)

#Visualização
regioes202205_df

,regional_office_name,chemical_substance_bnf_descr,quantity
0,NORTH WEST,Enteral nutrition,352008476.0
1,NORTH WEST,Emollients,48532489.0
2,NORTH WEST,Other emollient preparations,41137225.0
3,NORTH WEST,Other food for special diet preparations,37539283.0
4,NORTH WEST,Alginic acid compound preparations,30408395.0
...,...,...,...
115,SOUTH EAST,Colecalciferol,17965452.0
116,SOUTH EAST,Alginic acid compound preparations,17533977.0
117,SOUTH EAST,Levothyroxine sodium,17107065.0
118,SOUTH EAST,Lactulose,16274619.0


##### Extraindo dados do mês 6 do ano de 2022

In [8]:
#Selecionando o mês para extrair os dados
nome_dataset = 'EPD_202206'

#Criação do dataset em que será concatenado
regioes202206_df = pd.DataFrame()

#for loop para extrair cada dataset regional e então concatená-los
for i in range(0, len(todas_regioes_df)) :
   
   #Query para uma região
   query_uma_regiao = "SELECT " \
                           "regional_office_name, " \
                           "chemical_substance_bnf_descr, " \
                           "sum(total_quantity) as quantity " \
                        "FROM " \
                           f"`{nome_dataset}` " \
                        "WHERE " \
                           "regional_office_name = " f"'{todas_regioes_df.regional_office_name[i]}' " \
                        "GROUP BY " \
                           "regional_office_name, " \
                           "chemical_substance_bnf_descr " \
                        "ORDER BY " \
                           "quantity DESC " \
                        "LIMIT 15"

   #criação da url de requisição ao API
   uma_regiao_api_call = f"{base_endpoint}" \
                           f"{action_method}" \
                           "resource_id=" \
                           f"{nome_dataset}" \
                           "&" \
                           "sql=" \
                           f"{urllib.parse.quote(query_uma_regiao)}"

   #Resultado em JSON temporário
   json_uma_regiao = requests.get(uma_regiao_api_call).json()

   #Extração apenas dos resultados do JSON e transformação em um dataframe
   uma_regiao_df  = pd.json_normalize(json_uma_regiao['result']['result']['records'])

   #Concatenação dos dataframes de região
   frames = [regioes202206_df, uma_regiao_df]
   regioes202206_df = pd.concat(frames, sort = False)

#Arrumando o índice
regioes202206_df = regioes202206_df.reset_index(drop = True)

#Visualização
regioes202206_df

,regional_office_name,chemical_substance_bnf_descr,quantity
0,NORTH WEST,Enteral nutrition,351905747.0
1,NORTH WEST,Emollients,47989129.0
2,NORTH WEST,Other emollient preparations,40477476.0
3,NORTH WEST,Other food for special diet preparations,34659858.0
4,NORTH WEST,Alginic acid compound preparations,31421966.0
...,...,...,...
115,SOUTH EAST,Alginic acid compound preparations,18106519.0
116,SOUTH EAST,Colecalciferol,17199315.0
117,SOUTH EAST,Levothyroxine sodium,16487768.0
118,SOUTH EAST,Lactulose,15864135.0


##### Extraindo dados do mês 7 do ano de 2022

In [9]:
#Selecionando o mês para extrair os dados
nome_dataset = 'EPD_202207'

#Criação do dataset em que será concatenado
regioes202207_df = pd.DataFrame()

#for loop para extrair cada dataset regional e então concatená-los
for i in range(0, len(todas_regioes_df)) :
   
   #Query para uma região
   query_uma_regiao = "SELECT " \
                           "regional_office_name, " \
                           "chemical_substance_bnf_descr, " \
                           "sum(total_quantity) as quantity " \
                        "FROM " \
                           f"`{nome_dataset}` " \
                        "WHERE " \
                           "regional_office_name = " f"'{todas_regioes_df.regional_office_name[i]}' " \
                        "GROUP BY " \
                           "regional_office_name, " \
                           "chemical_substance_bnf_descr " \
                        "ORDER BY " \
                           "quantity DESC " \
                        "LIMIT 15"

   #criação da url de requisição ao API
   uma_regiao_api_call = f"{base_endpoint}" \
                           f"{action_method}" \
                           "resource_id=" \
                           f"{nome_dataset}" \
                           "&" \
                           "sql=" \
                           f"{urllib.parse.quote(query_uma_regiao)}"

   #Resultado em JSON temporário
   json_uma_regiao = requests.get(uma_regiao_api_call).json()

   #Extração apenas dos resultados do JSON e transformação em um dataframe
   uma_regiao_df  = pd.json_normalize(json_uma_regiao['result']['result']['records'])

   #Concatenação dos dataframes de região
   frames = [regioes202207_df, uma_regiao_df]
   regioes202207_df = pd.concat(frames, sort = False)

#Arrumando o índice
regioes202207_df = regioes202207_df.reset_index(drop = True)

#Visualização
regioes202207_df

,regional_office_name,chemical_substance_bnf_descr,quantity
0,NORTH WEST,Enteral nutrition,351184419.0
1,NORTH WEST,Emollients,46612845.0
2,NORTH WEST,Other emollient preparations,39373124.0
3,NORTH WEST,Other food for special diet preparations,34980650.0
4,NORTH WEST,Alginic acid compound preparations,31100892.0
...,...,...,...
115,SOUTH EAST,Alginic acid compound preparations,17642513.0
116,SOUTH EAST,Colecalciferol,16921961.3
117,SOUTH EAST,Levothyroxine sodium,16601375.0
118,SOUTH EAST,Lactulose,15596483.0


##### Concatenação e criação do arquivo final

In [10]:
#Concatenação dos dataframes de todos os meses
frames = [regioes202205_df, regioes202206_df, regioes202207_df]
regioes_df = pd.concat(frames, sort = False)

#Arrumando o índice
regioes_df = regioes_df.reset_index(drop = True)


In [11]:
#Dataframe será concatenado os dados das 10 substâncias químicas para cada região
regioestop10_df = pd.DataFrame()

#For loop para extrair as 10 substâncias entre as 15 em cada região e então concatená-las
for i in range(0, len(todas_regioes_df)) :
    
    #Dataframe temporário com todos os dados de cada região para todos os meses
    temp_reg = regioes_df.loc[regioes_df.regional_office_name == todas_regioes_df.regional_office_name[i]]
    
    #Dataframe temporário que agrupa as substâncias químicas, soma as quantidades dada a região e as ordena.
    temp_reg = temp_reg.groupby(['regional_office_name','chemical_substance_bnf_descr']).sum().sort_values('quantity', ascending=False).head(10)
    
    #Concatenação das regiões
    frames = [regioestop10_df, temp_reg]
    regioestop10_df = pd.concat(frames, sort=False)

#Visualização
regioestop10_df

quantity
regional_office_name chemical_substance_bnf_descr                            
NORTH WEST           Enteral nutrition                           1.055099e+09
                     Emollients                                  1.431345e+08
                     Other emollient preparations                1.209878e+08
                     Other food for special diet preparations    1.071798e+08
                     Alginic acid compound preparations          9.293125e+07
...                                                                       ...
SOUTH EAST           Other emollient preparations                7.607960e+07
                     Metformin hydrochloride                     7.541457e+07
                     Atorvastatin                                6.545958e+07
                     Omeprazole                                  5.808119e+07
                     Co-codamol (Codeine phosphate/paracetamol)  5.756270e+07

[80 rows x 1 columns]

In [12]:
#Exportação para um arquivo CSV
regioestop10_df.to_csv('tratados/regiaotop10.csv')

# 10 químicos mais custosos por mês

### Extração e concatenação dos 10 químicos mais custosos por mês

A questão pede por "produtos químico", e por esse motivo usou-se o "chemical_substance_bnf_descr", que descreve ingrediente ativo da droga, ignorando nomes comerciais ou dosagens.\
Usou-se a coluna "actual_cost" que é o valor do químico já com impostos, subsídios e custos de transporte e armazenamentos já agregados, pois fora entendido que melhor refletiria a ideia de custo.

In [13]:
#Lista com todos os meses a serem extraídos
meses = ['EPD_202205','EPD_202206','EPD_202207']

#Criação do dataset em que será concatenado
custosos_df = pd.DataFrame()

#for loop para extrair cada dataset mensal e então concatená-los
for i in range(0, len(meses)) :
   
   #Query para um mês
   query_um_mes = "SELECT " \
                           "chemical_substance_bnf_descr, " \
                           "sum(actual_cost) as cost " \
                        "FROM " \
                           f"`{meses[i]}` " \
                        "GROUP BY " \
                           "chemical_substance_bnf_descr " \
                        "ORDER BY " \
                           "cost DESC " \
                        "LIMIT 10"

   #criação da url de requisição ao API
   um_mes_api_call = f"{base_endpoint}" \
                           f"{action_method}" \
                           "resource_id=" \
                           f"{meses[i]}" \
                           "&" \
                           "sql=" \
                           f"{urllib.parse.quote(query_um_mes)}"

   #Resultado em JSON temporário
   json_um_mes = requests.get(um_mes_api_call).json()

   #Extração apenas dos resultados do JSON e transformação em um dataframe
   um_mes_df  = pd.json_normalize(json_um_mes['result']['result']['records'])

   #Concatenação dos dataframes mensais
   frames = [custosos_df, um_mes_df]
   custosos_df = pd.concat(frames, sort = False)

#Agrupando e ordenando os 10 químicos mais custosos em todos os meses
custosos_df = custosos_df.groupby('chemical_substance_bnf_descr').sum().sort_values('cost', ascending=False).head(10)

#Arrumando o índice
custosos_df = custosos_df.reset_index(drop = False)

#Visualização
custosos_df

,chemical_substance_bnf_descr,cost
0,Apixaban,1.002657e+08
1,Enteral nutrition,7.329262e+07
2,Beclometasone dipropionate,7.218497e+07
3,Rivaroxaban,5.704191e+07
4,Catheters,3.908472e+07
5,Detection Sensor Interstitial Fluid/Gluc,3.317934e+07
6,Budesonide,2.951008e+07
7,Wound Management & Other Dressings,2.847481e+07
8,Glucose blood testing reagents,1.880945e+07
9,Edoxaban,1.868393e+07


### Exportando para CSV

In [14]:
#exportação para CSV dos químicos mais custosos em todos os meses
custosos_df.to_csv('tratados/custotop10.csv')

# Químicos mais prescritos

Nesta questão pediu-se as prescrições mais comuns, por esse motivo foi usada a coluna "bnf_description" que descreve o produto farmacológico, contendo sua dosagem.\
Pelo mesmo motivo preferiu-se o uso da coluna "item" pois ela indica melhor o número de prescrição, ignorando prescrições duplas (por exemplo: se na mesma receita tiver 2 cx de um medicamento com 12 comprimidos cada, conta apenas como 1)

In [15]:
#Lista com todos os meses a serem extraídos
meses = ['EPD_202205','EPD_202206','EPD_202207']

#Criação do dataset em que será concatenado
prescritos_df = pd.DataFrame()

#for loop para extrair cada dataset mensal e então concatená-los
for i in range(0, len(meses)) :
   
   #Query para um mês
   query_um_mes = "SELECT " \
                           "bnf_description, " \
                           "items " \
                        "FROM " \
                           f"`{meses[i]}` " \
                        "LIMIT 10"

   #criação da url de requisição ao API
   um_mes_api_call = f"{base_endpoint}" \
                           f"{action_method}" \
                           "resource_id=" \
                           f"{meses[i]}" \
                           "&" \
                           "sql=" \
                           f"{urllib.parse.quote(query_um_mes)}"

   #Resultado em JSON temporário
   json_um_mes = requests.get(um_mes_api_call).json()

   #Extração apenas dos resultados do JSON e transformação em um dataframe
   um_mes_df  = pd.json_normalize(json_um_mes['result']['result']['records'])

   #Concatenação dos dataframes mensais
   frames = [prescritos_df, um_mes_df]
   prescritos_df = pd.concat(frames, sort = False)

#Agrupando e ordenando os 10 medicamentos mais prescritos em todos os meses
prescritos_df = prescritos_df.groupby('bnf_description').sum().sort_values('items', ascending=False).head(10)

#Arrumando o índice
prescritos_df = prescritos_df.reset_index(drop = False)

#Visualização
prescritos_df

,bnf_description,items
0,Lansoprazole 15mg gastro-resistant capsules,77
1,Amitriptyline 10mg tablets,47
2,Venlafaxine 150mg modified-release capsules,18
3,Gabapentin 100mg capsules,13
4,Furosemide 20mg tablets,10
5,Zopiclone 7.5mg tablets,7
6,Tramadol 50mg capsules,4
7,Vagifem 10microgram vaginal tablets,4
8,Lymecycline 408mg capsules,3
9,Zolpidem 5mg tablets,3


In [16]:
prescritos_df.to_csv('tratados/prescritos.csv')

# Mais prescritos por prescriber

Foi usado a coluna "practice_name" para identificar o prescriber.\
Por demandar "produto químico" foi usada a coluna "chemical_substance_bnf_descr" para identificá-lo.\
Por demandar número de prescrição foi usado a coluna "items".\

Neste há o problema de descartar valores iguais para diferentes medicamentos com a mesma quantidade de prescrições
por exemplo:\
se uma prescriber só receitou vários medicamentos apenas 1 vez, essa pipeline deixará apenas um desses medicamentos.\
Com maior tempo isso pode ser corrigido.

In [17]:
#Lista com todos os meses a serem extraídos
meses = ['EPD_202205','EPD_202206','EPD_202207']

In [18]:
#lista com todas as prescribers
prescribers_df = pd.DataFrame()

#for loop para lista de todos as prescribers
for i in range(0, len(meses)):

    #Query para um mês
    query_um_mes = "SELECT " \
                   "T.practice_name, " \
                   "T.chemical_substance_bnf_descr, " \
                   "T.items " \
                   "FROM " \
                   f"`{meses[i]}` as T " \
                   "WHERE " \
                   f"T.items = ( SELECT MAX(items) FROM `{meses[i]}` WHERE practice_name = T.practice_name)" 

    #criação da url de requisição ao API
    um_mes_api_call = f"{base_endpoint}" \
                           f"{action_method}" \
                           "resource_id=" \
                           f"{meses[i]}" \
                           "&" \
                           "sql=" \
                           f"{urllib.parse.quote(query_um_mes)}"

    #Resultado em JSON temporário
    json_um_mes = requests.get(um_mes_api_call).json()

    #Extração apenas dos resultados do JSON e transformação em um dataframe
    um_mes_df  = pd.json_normalize(json_um_mes['result']['result']['records'])

    #dataframes de prescribers
    frames = [prescribers_df, um_mes_df]
    prescribers_df = pd.concat(frames, sort= False)

#limpeza e tratamento dos dados concatenados
prescribers_df = prescribers_df.groupby(['practice_name','chemical_substance_bnf_descr'])['items'].sum().reset_index().sort_values(
    ['practice_name', 'items'], ascending=False).drop_duplicates(
    ['practice_name']).sort_values('practice_name')

#arrumando o índice
prescribers_df = prescribers_df.reset_index(drop = True)

#visualização
prescribers_df

,practice_name,chemical_substance_bnf_descr,items
0,(FRACTURE CLINIC) NORTH OOH,Flucloxacillin sodium,662
1,(IRLAM) SALFORD CARE CTRS MEDICAL PRACTI,Omeprazole,455
2,(OUT PATIENT DEPARTMENT) NORTH OOH,Glycopyrronium bromide,4
3,0-19 EAST CHESHIRE HEALTH VISITORS,Emollients,4
4,0-19 PUBLIC HEALTH SERVICE HARTLEPOOL,Miconazole nitrate,5
...,...,...,...
8716,YOUR HEALTHCARE NON MED PRES,Night Drainage Bags,43
8717,YOXALL,Atorvastatin,714
8718,ZAIN MEDICAL CENTRE,Paracetamol,42
8719,ZAMAN,Atorvastatin,412


## Exportando para CSV

In [19]:
prescribers_df.to_csv('tratados/prescribers.csv')

# Prescribers adicionadas

In [20]:
#Lista com todos os meses a serem extraídos
meses = ['EPD_202206','EPD_202207']

Há duas queries que consultam a quantidade de prescribers prescritoras, uma para o mês 6 e outra para o mês 7.\
Com o resultado de cada query foi feito a diferença. Que é o acréscimo de prescribers para o último mês

In [21]:
#Query para o mês 6 para quantidade de prescribers
query_qtent202206 = "SELECT " \
                "count(distinct(practice_name))" \
                "FROM " \
                f"`{meses[0]}`" \

#criação da url de requisição ao API
qtent202206_api_call = f"{base_endpoint}" \
                        f"{action_method}" \
                        "resource_id=" \
                        f"{meses[0]}" \
                        "&" \
                        "sql=" \
                        f"{urllib.parse.quote(query_qtent202206)}"

#Resultado em JSON
json_qtent202206 = requests.get(qtent202206_api_call).json()

#Extração apenas dos resultados do JSON e transformação em um dataframe
qtent202206_df  = pd.json_normalize(json_qtent202206['result']['result']['records'])

#Query para o mês 7 para quantidade de prescribers
query_qtent202207 = "SELECT " \
                "count(distinct(practice_name))" \
                "FROM " \
                f"`{meses[1]}`" \

#criação da url de requisição ao API
qtent202207_api_call = f"{base_endpoint}" \
                        f"{action_method}" \
                        "resource_id=" \
                        f"{meses[1]}" \
                        "&" \
                        "sql=" \
                        f"{urllib.parse.quote(query_qtent202207)}"

#Resultado em JSON
json_qtent202207 = requests.get(qtent202207_api_call).json()

#Extração apenas dos resultados do JSON e transformação em um dataframe
qtent202207_df  = pd.json_normalize(json_qtent202207['result']['result']['records'])

In [22]:
#Diferença entre o mês 7 e 6, o resultado é o acréscimo de prescritores.
diff = qtent202207_df.f0_[0] - qtent202206_df.f0_[0]

#Exportando a diferença para um txt
with open(f'tratados/aumentoPrescriber.txt', 'w') as f:
    f.write(f'Houve um acréscimo de {diff} no mês 7 em relação ao 6, em 2022')

# Prescribers em mais de uma região

Para resolução dessa foi usada a coluna "practice_name" para identificar os prescribers, em que os dados foram agrupados.\
Foi contado a quantidade de vezes que "regional_office_name" aparece em cada grupo e então excluído os iguais a 1

In [23]:
#Lista com todos os meses a serem extraídos
meses = ['EPD_202205','EPD_202206','EPD_202207']

#Criação do dataset em que será concatenado
presregiao_df = pd.DataFrame()

#for loop para extrair cada dataset mensal e então concatená-los
for i in range(0, len(meses)) :
   
   #Query para um mês
   query_um_mes = "SELECT " \
                           "practice_name, " \
                           "COUNT(DISTINCT(regional_office_name)) as regqt " \
                        "FROM " \
                           f"`{meses[i]}` " \
                        "GROUP BY " \
                            "practice_name"

   #criação da url de requisição ao API
   um_mes_api_call = f"{base_endpoint}" \
                           f"{action_method}" \
                           "resource_id=" \
                           f"{meses[i]}" \
                           "&" \
                           "sql=" \
                           f"{urllib.parse.quote(query_um_mes)}"

   #Resultado em JSON temporário
   json_um_mes = requests.get(um_mes_api_call).json()

   #Extração apenas dos resultados do JSON e transformação em um dataframe
   um_mes_df  = pd.json_normalize(json_um_mes['result']['result']['records'])

   #Concatenação dos dataframes mensais
   frames = [presregiao_df, um_mes_df]
   presregiao_df = pd.concat(frames, sort = False)

#arrumando o dataframe para facilitar a leitura
presregiao_df = presregiao_df[['practice_name', 'regqt']]

#excluindo linhas em que as regiões são iguais a 1
presregiao_df = presregiao_df[presregiao_df.regqt > 1]

#Ordenando os valores
presregiao_df = presregiao_df.sort_values('regqt', ascending=False)

#Arrumando o índice
presregiao_df = presregiao_df.reset_index(drop = True)

In [24]:
#Visualizando
presregiao_df

,practice_name,regqt
0,UNIDENTIFIED DOCTORS,8
1,UNIDENTIFIED DOCTORS,8
2,UNIDENTIFIED DOCTORS,8
3,VILLAGE SURGERY,7
4,VILLAGE SURGERY,7
...,...,...
654,ASHVILLE SURGERY,2
655,THE PARK MEDICAL CENTRE,2
656,MANOR MEDICAL PRACTICE,2
657,THE JUNCTION SURGERY,2


In [25]:
#Exportando para CSV
presregiao_df.to_csv('tratados/precriberPorRegiao.csv')

# Preço médio dos prescritos

Usou-se a coluna "bnf_description" para identificar o produto, em que os dados foram agrupados.\
Foi usada a soma da coluna "total_quantity" para que o preço médio fosse por dose/comprimido.\
Foi usada a soma da coluna "actual_cost" para ter um valor do preço mais preciso.\
Foi dividido então a somatória do custo com a somatoria da quantidade, desta forma, foi calculado o preço médio por dose/comprimido.

In [26]:
#Lista com todos os meses a serem extraídos
meses = ['EPD_202205','EPD_202206','EPD_202207']

#Criação do dataset em que será concatenado
medio_df = pd.DataFrame()

#Query para um mês
query_um_mes = "SELECT " \
                        "distinct(bnf_description), " \
                        "sum(total_quantity) as qt , " \
                        "sum(actual_cost) as cost " \
                     "FROM " \
                        f"`{meses[2]}` " \
                     "GROUP BY " \
                           "bnf_description"

#criação da url de requisição ao API
um_mes_api_call = f"{base_endpoint}" \
                        f"{action_method}" \
                        "resource_id=" \
                        f"{meses[2]}" \
                        "&" \
                        "sql=" \
                        f"{urllib.parse.quote(query_um_mes)}"

#Resultado em JSON temporário
json_um_mes = requests.get(um_mes_api_call).json()

#Extração apenas dos resultados do JSON e transformação em um dataframe
medio_df  = pd.json_normalize(json_um_mes['result']['result']['records'])

#Criando a coluna de média
medio_df['mean'] = medio_df.cost/medio_df.qt 

#excluindo colunas de custo e quantidade
medio_df.drop(['cost','qt'], axis = 1, inplace= True)

#Arrumando o índice
medio_df = medio_df.reset_index(drop = True)

In [27]:
#Visualização
medio_df

,bnf_description,mean
0,Morphine sulfate 10mg/1ml solution for injecti...,1.564924
1,Buprenorphine 8mg sublingual tablets sugar free,0.656540
2,Brimonidine 3mg/g gel,1.050414
3,Cetraben ointment,0.013317
4,Co-amoxiclav 125mg/31mg/5ml oral suspension su...,0.020291
...,...,...
21984,SenSura Click two piece ostomy system colostom...,1.581771
21985,Hydrochlorothiazide 5mg/5ml oral liquid,0.192659
21986,Xaluprine 20mg/ml oral suspension,1.589646
21987,Quinine bisulfate 300mg/5ml oral suspension,0.282879


In [28]:
#Exportação para documento CSV
medio_df.to_csv('tratados/medio.csv')

# Prescrição de maior valor por prescriber

In [29]:
#Lista com todos os meses a serem extraídos
meses = ['EPD_202205','EPD_202206','EPD_202207']

#dataframe que receberá os dados
maiorvalor_df = pd.DataFrame()

#for loop para lista de todos as prescribers
for i in range(0, len(meses)):

    #Query para um mês
    query_um_mes = "SELECT " \
                   "T.practice_name, " \
                   "T.bnf_description, " \
                   "T.actual_cost " \
                   "FROM " \
                   f"`{meses[i]}` as T " \
                   "WHERE " \
                   f"T.actual_cost = ( SELECT MAX(actual_cost) FROM `{meses[i]}` WHERE practice_name = T.practice_name)" 

    #criação da url de requisição ao API
    um_mes_api_call = f"{base_endpoint}" \
                           f"{action_method}" \
                           "resource_id=" \
                           f"{meses[i]}" \
                           "&" \
                           "sql=" \
                           f"{urllib.parse.quote(query_um_mes)}"

    #Resultado em JSON temporário
    json_um_mes = requests.get(um_mes_api_call).json()

    #Extração apenas dos resultados do JSON e transformação em um dataframe
    um_mes_df  = pd.json_normalize(json_um_mes['result']['result']['records'])

    #dataframes de prescribers
    frames = [maiorvalor_df, um_mes_df]
    maiorvalor_df = pd.concat(frames, sort= False)

#Arrumando o índice
maiorvalor_df = maiorvalor_df.reset_index(drop = True)


In [30]:
#visualização
maiorvalor_df

,practice_name,bnf_description,actual_cost
0,TISSUE VIABILITY NURSING SERVICE,BioBag 100 4cm x 5cm dressing kit,1016.31561
1,ELTON REHABILITATION UNIT,Oxycodone 5mg modified-release tablets,35.14982
2,HEREFORDSHIRE MH GENERAL,Melatonin 3mg capsules,109.25730
3,CHAPELTOWN FAMILY SURGERY,Hydrocortisone 2.5mg tablets,1452.80795
4,CONISBROUGH MEDICAL PRACTICE,Norditropin FlexPro 10mg/1.5ml inj pre-filled ...,795.94900
...,...,...,...
25315,FOUNDRY LANE SURGERY,Trelegy Ellipta 92microg/55microg/22microg/dos...,2164.32101
25316,ST.ANDREW'S HOUSE SURGERY,Trelegy Ellipta 92microg/55microg/22microg/dos...,2247.56413
25317,TRINITY & BOWTHORPE MEDICAL PRACTICE,Trelegy Ellipta 92microg/55microg/22microg/dos...,1706.48388
25318,TONG MEDICAL PRACTICE,Trelegy Ellipta 92microg/55microg/22microg/dos...,3371.34619


In [31]:
#Exportando para CSV
maiorvalor_df.to_csv('tratados/maiorValorPorPrescriber.csv')

# Preparação para Automação

Esta etapa do processo não deverá ser executado todos os meses, e está aqui apenas para preparação do processo de automatização, ou seja, é executada apenas uma vez.

In [32]:
#Obtendo lista de Datasets
#datasets_response = requests.get(base_endpoint + "package_show?id=english-prescribing-data-epd").json()

#listando todos os metadados dos datasets
#lista_df  = pd.json_normalize(datasets_response['result']['resources'])

#selecionando apenas os nomes dos datasets
#listameses = lista_df['name']

#Transformando lista em formato panda series para python list
#listameses = listameses.tolist()

#Deletando o último mês para ficar de acordo com os requisitos e então poder testar a automação
#(esta linha serve apenas para o desafio e só existiria nesse caso para testar)
#listameses.pop()

# Escrevendo lista de datasets já tratados
#with open('lista_tratados.txt','w') as lista_tratados:
#    for values in listameses:
#        lista_tratados.write('%s\n' % values)